# Feature engineering, xgboost
https://www.kaggle.com/dlarionov/feature-engineering-xgboost


-　DateNum作成 'date_block_num',→YYYYMMで代用
- 祝日フラグ
- month
- days
- アイテムカテゴリ（デフォルト)-> カテゴリCategory、ジャンルType
- 月の売り上げ
- 月の売り上げラグ 1,2,3,6,12
- アイテムあたりの「週Or日」平均販売個数ラグ1，2，3，6，12　　'date_item_avg_item_cnt_lag_1',
- ショップあたり「週Or日」平均販売個数ラグ1，2，3，6，12　　'date_shop_avg_item_cnt_lag_1',
- アイテムカテゴリあたり「週Or日」平均販売個数ラグ1，2，3，'date_cat_avg_item_cnt_lag_1',
- カテゴリあたり「週Or日」平均販売個数ラグ1，2，3，'date_cat_avg_item_cnt_lag_1',
- カテゴリ/ショップあたり[月] 平均販売個数ラグ1，2，3，
- ジャンル　あたり[月] 平均販売個数ラグ1，2，3，
- ジャンル/ショップ　あたり[月] 平均販売個数ラグ1，2，3，
- ASP　月当たり
- 価格下落率
- 最終販売日
- 初販売日
- ショップあたりの最終販売日
- ショップあたりの初販売日
    'delta_price_lag',


In [1]:
import numpy as np
import pandas as pd
import time
import matplotlib.pyplot as plt
import seaborn as sns
%matplotlib inline

In [2]:
#データの読込
sales = pd.read_csv('./sales_history.csv')
test = pd.read_csv('./test.csv')
sales['日付'] = pd.to_datetime(sales['日付'])
base_df = pd.read_csv('./csv/date_block_base_df.csv')

In [3]:
#sales_total = pd.read_csv('sales_total.csv')

from datetime import date,timedelta
import jpholiday
%time
sales['is_holiday'] = sales['日付'].map(jpholiday.is_holiday).astype(int)

CPU times: user 5 µs, sys: 1e+03 ns, total: 6 µs
Wall time: 11.7 µs


In [6]:
sales

,日付,店舗ID,商品ID,商品価格,売上個数,is_holiday,年,月
0,2018-01-01,9,1000001,420,1.0,1,2018,1
1,2018-01-01,6,1000001,420,1.0,1,2018,1
2,2018-01-01,10,1000001,420,1.0,1,2018,1
3,2018-01-01,0,1000017,250,1.0,1,2018,1
4,2018-01-01,1,1000017,250,1.0,1,2018,1
...,...,...,...,...,...,...,...,...
1119565,2019-10-31,8,3400006,2700,1.0,0,2019,10
1119566,2019-10-31,7,3400009,1020,1.0,0,2019,10
1119567,2019-10-31,7,3400012,510,1.0,0,2019,10
1119568,2019-10-31,4,3500001,340,1.0,0,2019,10


In [7]:
#sales_historyのデータを月次にリサンプリング
sales['年'] = sales['日付'].dt.year
sales['月'] = sales['日付'].dt.month
#sales['YYYYMM'] = sales['日付'].dt.strftime('%Y%m')

sales_month = sales.groupby(
    ['日付','年', '月', '店舗ID', '商品ID'], as_index=False).agg({
    '商品価格':'mean','売上個数':'sum','is_holiday':'sum'})
sales_month

,日付,年,月,店舗ID,商品ID,商品価格,売上個数,is_holiday
0,2018-01-01,2018,1,0,1000017,250.0,1.0,1
1,2018-01-01,2018,1,0,1000032,680.0,1.0,1
2,2018-01-01,2018,1,0,1000034,680.0,2.0,1
3,2018-01-01,2018,1,0,1000044,510.0,1.0,1
4,2018-01-01,2018,1,0,1000046,680.0,2.0,1
...,...,...,...,...,...,...,...,...
1062493,2019-10-31,2019,10,17,2200008,1360.0,1.0,0
1062494,2019-10-31,2019,10,17,2600077,3400.0,1.0,0
1062495,2019-10-31,2019,10,17,2800001,1190.0,1.0,0
1062496,2019-10-31,2019,10,17,3200024,4230.0,1.0,0


In [8]:
pd.merge(sales_month,base_df, on=['年','月'],how='left')

,日付,年,月,店舗ID,商品ID,商品価格,売上個数,is_holiday,date_block_num
0,2018-01-01,2018,1,0,1000017,250.0,1.0,1,0
1,2018-01-01,2018,1,0,1000032,680.0,1.0,1,0
2,2018-01-01,2018,1,0,1000034,680.0,2.0,1,0
3,2018-01-01,2018,1,0,1000044,510.0,1.0,1,0
4,2018-01-01,2018,1,0,1000046,680.0,2.0,1,0
...,...,...,...,...,...,...,...,...,...
1062493,2019-10-31,2019,10,17,2200008,1360.0,1.0,0,21
1062494,2019-10-31,2019,10,17,2600077,3400.0,1.0,0,21
1062495,2019-10-31,2019,10,17,2800001,1190.0,1.0,0,21
1062496,2019-10-31,2019,10,17,3200024,4230.0,1.0,0,21


In [9]:
#sales_month.to_csv('./csv/sales_month_date_block_holiday.csv',index=False)

In [13]:
#月次売上データから重点予測対象商品170点の商品のみを抽出
important_items = test[['商品ID', '店舗ID']]
predict_items_df = pd.merge(sales_month, important_items, on=['商品ID', '店舗ID'])

#2018-01 ~ 2019-10までの結合用のDataFrameを作成
month_2018 = np.arange(1, 13)
month_2019 = np.arange(1, 11)

df_2018 = pd.DataFrame(month_2018, columns=['月'])
df_2018['年'] = 2018

df_2019 = pd.DataFrame(month_2019, columns=['月'])
df_2019['年'] = 2019

#欠損値補完の準備

"""
base_df: 結合の土台にするDataFrame 【dataframe】
month_sales_df: 補完後のDataFrameを追加していくためのDataFrame 【dataframe】
shop_id: 0~17の店舗ID 【array】
items: 重点予測対象の商品170点の商品ID 【array】
tmp_df: 計算のために使う一時的なDataFrame 【dataframe】
"""

#base_df = pd.concat([df_2018, df_2019], axis=0)[['年', '月']]
month_sales_df = pd.DataFrame()

,date_block_num,年,月
0,0,2018,1
1,1,2018,2
2,2,2018,3
3,3,2018,4
4,4,2018,5
5,5,2018,6
6,6,2018,7
7,7,2018,8
8,8,2018,9
9,9,2018,10


170

In [22]:
shop_id = np.arange(0, 18)
items = important_items['商品ID'].unique()

#欠損値補完処理(処理時間計測開始)
start_time = time.time()

for id in shop_id:
    for item in items:
        tmp_df = base_df.copy()
        tmp_df['店舗ID'] = id
        tmp_df['商品ID'] = item
        item_df = predict_items_df[(predict_items_df['商品ID'] == item) & 
                                   (predict_items_df['店舗ID'] == id)].drop(columns='店舗ID')
        tmp_df = pd.merge(tmp_df, item_df, on=[
            '年', '月', '商品ID'], how='left').fillna(0)
        month_sales_df = month_sales_df.append(tmp_df)

end_time = time.time()
#計測終了

elasped_time = end_time - start_time

print('処理にかかった時間: {:.3f}秒'.format(elasped_time))

処理にかかった時間: 19.608秒


In [23]:
month_sales_df

,date_block_num,年,月,店舗ID,商品ID,日付,商品価格,売上個数,is_holiday
0,0,2018,1,0,1000001,2018-01-06 00:00:00,420.0,1.0,0.0
1,0,2018,1,0,1000001,2018-01-07 00:00:00,420.0,3.0,0.0
2,0,2018,1,0,1000001,2018-01-20 00:00:00,420.0,1.0,0.0
3,0,2018,1,0,1000001,2018-01-28 00:00:00,420.0,1.0,0.0
4,1,2018,2,0,1000001,2018-02-07 00:00:00,420.0,1.0,0.0
...,...,...,...,...,...,...,...,...,...
17,17,2019,6,17,3500001,0,0.0,0.0,0.0
18,18,2019,7,17,3500001,0,0.0,0.0,0.0
19,19,2019,8,17,3500001,0,0.0,0.0,0.0
20,20,2019,9,17,3500001,0,0.0,0.0,0.0


In [29]:
#month_sales_df.to_csv('./csv/sales_filled_month_date_block_holiday.csv',index=False)
sales_history = month_sales_df

In [30]:
category_names = pd.read_csv('category_names.csv')
category_names.columns = ['ItemCategoryId', 'ItemCategoryName']

item_categories = pd.read_csv('item_categories.csv')
item_categories.columns = ['ItemId', 'ItemCategoryId']

#sales_history = month_sales_df
#sales_history = pd.read_csv('./csv/sales_filled_month_date_block_holiday.csv')

sales_history.columns = ['date_block_num', 'YYYY','MM','StoreId', 'ItemId','Date',
                         'ItemPrice', 'SoldNum','is_holiday' ]
#sales_history['Date'] = pd.to_datetime(sales_history['Date'])
sales_history

,date_block_num,YYYY,MM,StoreId,ItemId,Date,ItemPrice,SoldNum,is_holiday
0,0,2018,1,0,1000001,2018-01-06 00:00:00,420.0,1.0,0.0
1,0,2018,1,0,1000001,2018-01-07 00:00:00,420.0,3.0,0.0
2,0,2018,1,0,1000001,2018-01-20 00:00:00,420.0,1.0,0.0
3,0,2018,1,0,1000001,2018-01-28 00:00:00,420.0,1.0,0.0
4,1,2018,2,0,1000001,2018-02-07 00:00:00,420.0,1.0,0.0
...,...,...,...,...,...,...,...,...,...
17,17,2019,6,17,3500001,0,0.0,0.0,0.0
18,18,2019,7,17,3500001,0,0.0,0.0,0.0
19,19,2019,8,17,3500001,0,0.0,0.0,0.0
20,20,2019,9,17,3500001,0,0.0,0.0,0.0


In [32]:
test = pd.read_csv('test.csv')
test.columns = ['index', 'ItemId', 'StoreId']
#item_categories['ItemCategoryId'].unique()
#category_names.query('ItemCategoryName.str.contains("")')
'''170品目のリスト'''
test_item = test['ItemId'].unique()
test_item = pd.DataFrame({'ItemId':test_item})
#test_item
'''カテゴリ名と連結'''
ttest = pd.merge(test,item_categories, on='ItemId')
ttest = pd.merge(ttest, category_names,on='ItemCategoryId')
print('予測データの品目数: ',ttest['ItemId'].nunique(),'\n',
      '訓練データの品目数: ',sales_history['ItemId'].nunique(),'\n',
      '予測データのカテゴリ数: ',item_categories['ItemCategoryId'].nunique(),'\n',
      '訓練データのカテゴリ数: ',ttest['ItemCategoryId'].nunique(),'\n',
     '売上の総数: ',sales_history['SoldNum'].sum(), '\n',
     '')

予測データの品目数:  170 
 訓練データの品目数:  170 
 予測データのカテゴリ数:  26 
 訓練データのカテゴリ数:  26 
 売上の総数:  50595.0 
 


# カテゴリ・ジャンル作成
# sklearn Labelencoding

In [33]:
from sklearn import preprocessing
le = preprocessing.LabelEncoder()
#ItemCategoryNameを　「ー」で分割
cat_type = category_names['ItemCategoryName'].str.split('-',expand=True)
#カラム名変更
cat_type['Category'] = cat_type[0]
cat_type['Type'] = cat_type[1]
# ItemCategoryId追加
cat_type =pd.concat([cat_type,category_names['ItemCategoryId']],axis=1)
cat_type.rename(columns={0:'CatName',1:'TypeName'},inplace=True)
'''PCゲームをゲームに'''
cat_type.at[1,'Category'] = 'ゲーム'
cat_type.at[1,'CatName'] = 'ゲーム'
''' Label Encoding'''
cat_type.Category = le.fit_transform(cat_type['Category'].values)
cat_type.Type = le.fit_transform(cat_type['Type'].values)

'''testデータにカテゴリ特徴追加'''
pd.merge(ttest,cat_type,on='ItemCategoryId').drop(['CatName','TypeName'],axis=1)

'''全売り上げデータにカテゴリ特徴追加'''
sales_total = pd.merge(sales_history,item_categories,on='ItemId')
sales_total = pd.merge(sales_total,cat_type,on='ItemCategoryId')
#sales_total.groupby('ItemCategoryName').count()
sales_total

,date_block_num,YYYY,MM,StoreId,ItemId,Date,ItemPrice,SoldNum,is_holiday,ItemCategoryId,CatName,TypeName,Category,Type
0,0,2018,1,0,1000001,2018-01-06 00:00:00,420.0,1.0,0.0,100,映画,DVD,5,1
1,0,2018,1,0,1000001,2018-01-07 00:00:00,420.0,3.0,0.0,100,映画,DVD,5,1
2,0,2018,1,0,1000001,2018-01-20 00:00:00,420.0,1.0,0.0,100,映画,DVD,5,1
3,0,2018,1,0,1000001,2018-01-28 00:00:00,420.0,1.0,0.0,100,映画,DVD,5,1
4,1,2018,2,0,1000001,2018-02-07 00:00:00,420.0,1.0,0.0,100,映画,DVD,5,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
89071,17,2019,6,17,3500001,0,0.0,0.0,0.0,350,本,オーディオブック_タイプB,6,6
89072,18,2019,7,17,3500001,0,0.0,0.0,0.0,350,本,オーディオブック_タイプB,6,6
89073,19,2019,8,17,3500001,0,0.0,0.0,0.0,350,本,オーディオブック_タイプB,6,6
89074,20,2019,9,17,3500001,0,0.0,0.0,0.0,350,本,オーディオブック_タイプB,6,6


In [35]:
sales_total.shape

(89076, 14)

# 休日フラグ  sales_total.csvとして保存
https://upura.hatenablog.com/entry/2018/12/21/070000

In [34]:
#sales_total.to_csv('./csv/sales_total.csv',index=False)

In [37]:
sales_total = pd.read_csv('./csv/sales_170filled.csv')

(89076, 14)